In [1]:
import os
import sys
home_dir = "../../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd

In [2]:
from proteinbert import load_pretrained_model, tokenize_seqs
from proteinbert.tokenization import token_to_index, index_to_token

2023-06-09 09:08:45.034985: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-09 09:08:45.198999: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-09 09:08:45.934288: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-09 09:08:45.941475: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 09:08:48.789641: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [53]:
print(token_to_index["A"])
print(token_to_index["<START>"])
print(token_to_index["<END>"])
print(token_to_index["<PAD>"])
print(token_to_index["<OTHER>"]) # non-standard amino acids are mapped tho <OTHER> token

0
23
24
25
22


In [9]:
pretrained_model_generator, tokenizer = load_pretrained_model(local_model_dump_dir=home_dir+"models/proteinbert_brandes/cache/", local_model_dump_file_name="epoch_92400_sample_23500000.pkl")
model = pretrained_model_generator.create_model(1024)

In [11]:
seq = "MGWSCLVTGAGGLLGQRIVRLLVEEKELKEIRALDKAFRPELREEFSKLQNRTKLTVLEGDILDEPFLKRACQDVSVVIHTACIIDVFGVTHRESIMNVNVKGTQLLLEACVQASVPVFIYTSSIEVAGPNSYKEIIQNGHEEEPLENTWPTPYPYSKKLAEKAVLAANGWNLKNGDTLYTCALRPTYIYGEGGPFLSASINEALNNNGILSSVGKFSTVNPVYVGNVAWAHILALRALRDPKKAPSVRGQFYYISDDTPHQSYDNLNYILSKEFGLRLDSRWSLPLTLMYWIGFLLEVVSFLLSPIYSYQPPFNRHTVTLSNSVFTFSYKKAQRDLAYKPLYSWEEAKQKTVEWVGSLVDRHKETLKSKTQ"
# seq = "ABD"
print(len(seq))
print(seq[254-1])

372
Y


In [15]:
x = tokenizer.encode_X([seq], 1024) # '<START>'=23, '<END>'=24, <PAD>=25
logits, annotations = model(x) # shape=(n_seq=1, seq-len=1024, vocab_size=26), shape=(1, 8943)
logits = logits[0].numpy()
print(logits.shape)

mut_pos = 254 # 1-indexed
wt_aa,  mt_aa = "Y", "D"
wt_tok_idx = token_to_index["Y"]
mt_tok_idx = token_to_index["D"]
print(wt_tok_idx, mt_tok_idx)
wt_logit = logits[mut_pos][wt_tok_idx]
mt_logit = logits[mut_pos][mt_tok_idx]
mt_logit - wt_logit

(1024, 26)
21 2


-0.967368

In [17]:
def get_logits(seq):
    x = tokenizer.encode_X([seq], 1024) # '<START>'=23, '<END>'=24, <PAD>=25
    logits, annotations = model(x) # shape=(n_seq=1, seq-len=1024, vocab_size=26), shape=(1, 8943)
    logits = logits[0].numpy()
    print(logits.shape)
    return logits

In [23]:
wt_seq = seq
mt_seq = list(seq)
mt_seq[mut_pos-1] = mt_aa
mt_seq = "".join(mt_seq)
print(wt_seq[mut_pos-1], mt_seq[mut_pos-1])

mt_logits = get_logits(mt_seq)
mt_logits[mut_pos]

Y D
(1024, 26)


array([1.89526833e-03, 5.35019790e-04, 9.69498694e-01, 1.49636180e-03,
       1.82713103e-03, 1.99127500e-03, 1.64967985e-03, 1.35152461e-03,
       9.04428191e-04, 2.75730365e-03, 4.63545060e-04, 3.15711368e-03,
       9.35012999e-04, 1.15887646e-03, 1.56603870e-03, 2.60223052e-03,
       1.51586917e-03, 1.35588011e-14, 2.09474401e-03, 5.80061576e-04,
       6.46258400e-07, 2.01910897e-03, 3.34831503e-11, 1.37152101e-09,
       1.01854996e-08, 9.11189457e-11], dtype=float32)

In [22]:
wt_logits = get_logits(wt_seq)
wt_logits[mut_pos]

(1024, 26)


array([1.5726064e-03, 5.3479121e-04, 1.2859084e-03, 8.1308413e-04,
       1.2502178e-03, 1.7721958e-03, 3.2831961e-03, 8.8850153e-04,
       8.4672403e-04, 1.9653682e-03, 2.8045877e-04, 8.6312247e-03,
       7.1712804e-04, 1.0302503e-03, 1.1809288e-03, 2.2481875e-03,
       1.1844998e-03, 1.8173975e-15, 1.4552135e-03, 4.0520163e-04,
       4.5650094e-07, 9.6865392e-01, 4.2524578e-12, 3.9998063e-09,
       1.0908802e-09, 1.1904547e-10], dtype=float32)

In [26]:
import numpy as np
np.linalg.norm(mt_logits[mut_pos] - wt_logits[mut_pos])

1.3681573